In [1]:
import import_ipynb
import process_data
import math
import tensorflow as tf
import numpy as np
from functools import partial


LOSS_OUT_FILE = 'Epoch_Loss.txt'

process_data.process_wav()

# Learning rate
lr = 0.0001

# L2 regularization
l2 = 0.0001

inputs = 8000 #12348
hidden_1_size = 2048 #8400  
hidden_2_size = 1024 #3440  
hidden_3_size = 512 #2800  

# Change the epochs variable to define the 
# number of times we iterate through all our batches
epochs = 3 #100

# Change the batch_size variable to define how many songs to load per batch
batch_size = 32 #50  

# Change the batches variable to change the number of batches you want per epoch
batches = 1

# Define our placeholder with shape [?, 12348]
tf.compat.v1.disable_eager_execution()
X = tf.compat.v1.placeholder(tf.float32, shape=[None, inputs])
l2_regularizer = tf.keras.regularizers.l2(l2)

autoencoder_dnn = partial(tf.compat.v1.layers.dense, 
						activation = tf.nn.elu,
						kernel_initializer = tf.keras.initializers.VarianceScaling(),
						kernel_regularizer=  tf.keras.regularizers.l2(l2))

hidden_1 = autoencoder_dnn(X, hidden_1_size)
hidden_2 = autoencoder_dnn(hidden_1, hidden_2_size)
hidden_4 = autoencoder_dnn(hidden_2, hidden_3_size)
hidden_5 = autoencoder_dnn(hidden_4, hidden_2_size)
outputs =  autoencoder_dnn(hidden_5, inputs, activation=None)

reconstruction_loss = tf.reduce_mean(tf.square(outputs-X))
reg_loss = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.REGULARIZATION_LOSSES)
loss = tf.add_n([reconstruction_loss] + reg_loss)

optimizer = tf.compat.v1.train.AdamOptimizer(lr)
training_op = optimizer.minimize(loss)
init = tf.compat.v1.global_variables_initializer()

importing Jupyter notebook from process_data.ipynb
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [2]:
def next_batch(c_batch, batch_size, sess):
	ch1_arr = []
	ch2_arr = []
	#process_data.convert_mp3_to_wav()
	#print(c_batch,batch_size) #comment
	wav_arr_ch1, wav_arr_ch2, sample_rate = process_data.get_next_batch(c_batch, batch_size, sess)
	print('length of wav_arr_ch1 and wav_arr_ch2',len(wav_arr_ch1),len(wav_arr_ch2)) #comment

	for sub_arr in wav_arr_ch1:
		batch_size_ch1 = math.floor(len(sub_arr)/inputs)
		sub_arr = sub_arr[:(batch_size_ch1*inputs)]
		ch1_arr.append(np.array(sub_arr).reshape(batch_size_ch1, inputs))

	for sub_arr in  wav_arr_ch2:
		batch_size_ch2 = math.floor(len(sub_arr)/inputs)
		sub_arr = sub_arr[:(batch_size_ch2*inputs)]
		ch2_arr.append(np.array(sub_arr).reshape(batch_size_ch2, inputs))

	return np.array(ch1_arr), np.array(ch2_arr), sample_rate

In [3]:
##### Run training
with tf.compat.v1.Session() as sess:
	init.run()
	
	for epoch in range(epochs):
		epoch_loss = []
		print("Epoch: " + str(epoch))
		for i in range(batches):
			ch1_song, ch2_song, sample_rate = next_batch(i, batch_size, sess)
			print('ch1_song.shape:',ch1_song.shape,'ch2_song.shape:',ch2_song.shape) #comment
			total_songs = np.hstack([ch1_song, ch2_song])
			batch_loss = []

			for j in range(len(total_songs)):
				x_batch = total_songs[j]
				_, l = sess.run([training_op, loss], feed_dict={X:x_batch})
				batch_loss.append(l)
				print("Song loss: " + str(l))

			print("Curr Epoch: " + str(epoch) + " Curr Batch: " + str(i) + "/"+ str(batches))
			print("Batch Loss: " + str(np.mean(batch_loss)))
			epoch_loss.append(np.mean(batch_loss))

		print("Epoch Avg Loss: " + str(np.mean(epoch_loss)))

		if epoch % 1000 == 0:
			ch1_song_new, ch2_song_new, sample_rate_new = next_batch(2, 1, sess)
			print('ch1_song_new.shape:',ch1_song_new.shape,'ch2_song_new.shape:',ch2_song_new.shape) #comment
            
			x_batch = np.hstack([ch1_song_new, ch2_song_new])[0]
			print(x_batch.shape) #comment
			print("Sample rate: " + str(sample_rate_new))

			orig_song = []
			full_song = []
			evaluation = outputs.eval(feed_dict={X: x_batch})
			print("Output: " + str(evaluation))
			full_song.append(evaluation)
			orig_song.append(x_batch)

			# Merge the nested arrays
			orig_song = np.hstack(orig_song)
			full_song = np.hstack(full_song)

			# Compute and split the channels
			orig_song_ch1 = orig_song[:math.floor(len(orig_song)/2)]
			orig_song_ch2 = orig_song[math.floor(len(orig_song)/2):]
			full_song_ch1 = full_song[:math.floor(len(full_song)/2)]
			full_song_ch2 = full_song[math.floor(len(full_song)/2):]

			# Save both the untouched song and reconstructed song to the 'output' folder
			process_data.save_to_wav(full_song_ch1, full_song_ch2, sample_rate, orig_song_ch1, orig_song_ch2, epoch, 'output', sess)

Epoch: 0
length of file:  62
start_position: 0 end_position: 32
len(input): 1321967
Returning File: audio_wav/16.wav
len(input): 1323119
Returning File: audio_wav/17.wav
len(input): 1321967
Returning File: audio_wav/15.wav
len(input): 1321967
Returning File: audio_wav/29.wav
len(input): 1321967
Returning File: audio_wav/28.wav
len(input): 1321967
Returning File: audio_wav/14.wav
len(input): 1323119
Returning File: audio_wav/38.wav
len(input): 1321967
Returning File: audio_wav/10.wav
len(input): 1323119
Returning File: audio_wav/11.wav
len(input): 1321967
Returning File: audio_wav/39.wav
len(input): 1323119
Returning File: audio_wav/13.wav
len(input): 1321967
Returning File: audio_wav/12.wav
len(input): 1321967
Returning File: audio_wav/61.wav
len(input): 1321967
Returning File: audio_wav/49.wav
len(input): 1321967
Returning File: audio_wav/48.wav
len(input): 1323119
Returning File: audio_wav/60.wav
len(input): 1323119
Returning File: audio_wav/9.wav
len(input): 1321967
Returning File: 

Song loss: 30830.068
Song loss: 109650.42
Song loss: 15951.827
Song loss: 82274.695
Song loss: 55506.625
Song loss: 18410.307
Song loss: 38103.688
Song loss: 15709.615
Song loss: 3413.1873
Song loss: 17171.29
Song loss: 19328.166
Song loss: 11435.379
Song loss: 14084.794
Song loss: 11141.353
Song loss: 104307.92
Song loss: 59743.74
Song loss: 18954.46
Song loss: 12550.583
Song loss: 4617.835
Song loss: 11610.955
Song loss: 30180.688
Song loss: 5066.917
Song loss: 5594.1646
Song loss: 7361.611
Song loss: 42782.61
Song loss: 7642.6177
Song loss: 5640.0654
Song loss: 4769.815
Song loss: 54507.277
Song loss: 63264.387
Song loss: 1785.6655
Song loss: 13997.087
Curr Epoch: 2 Curr Batch: 0/1
Batch Loss: 28043.434
Epoch Avg Loss: 28043.434


<Figure size 432x288 with 0 Axes>